In [6]:
pip install html-to-json

Note: you may need to restart the kernel to use updated packages.


# Webscraper with BeautifulSoup

In [11]:
import requests
from bs4 import BeautifulSoup
import html2json
import json
from jsonpath_ng import jsonpath, parse

class scraper:
    def __init__(self, url):
        self.url = 'https://www.ulta.com/shop/makeup/face'
        self.json = {}

    
    def get_content(self):
        r = requests.get(self.url)
        return r.content
    
    def get_category(self):
        r = requests.get(self.url)
        soup = BeautifulSoup(r.content, "html.parser")
        content = soup.find_all(class_='CategoryCard')
        categories = {}
        for c in content:
            category_tag = c.find('a', class_='Link_Huge')
            category_name = category_tag.get_text()
            category_link = category_tag.get('href')
            self.json[category_name] = {}
            self.json[category_name]['url'] = category_link
            self.json[category_name]['products'] = []

            get_product_links(category_link)
        

    
    def get_product_links(self):
        soup = BeautifulSoup(self.content, "html.parser")
        content = soup.find_all(class_='ProductCard')
        products = {}

        for c in content:
            product_link_tag = c.find('a', class_='Link_Huge Link_Huge--secondary')
            product_link = product_link_tag.get('href')
            self.json[category_name]['products'].append(product_link)

            get_product_details(product_link)
    
    def get_product_details(product_link):
        r = requests.get(product_link)
        soup = BeautifulSoup(r.content, "html.parser")
        html_content = soup.find_all(class_='ProductHero__content')
        details = ''

        def extract_text_fields(json_data):
            """Extract all text fields using jsonpath-ng."""
            jsonpath_expression = parse('$.._value')
            matches = jsonpath_expression.find(json_data)
            return [match.value for match in matches]

        def html_to_text_json(html_content):
            """Convert HTML content to a nested JSON containing only text fields."""
            json_data = html_to_json.convert(html_content)
            text_fields = extract_text_fields(json_data)
            #print(json.dumps(json_data, indent=4))
            return text_fields

        # Convert HTML to text-only JSON
        text_json = html_to_text_json(str(html_content))

        # Print the JSON output
        json.dumps(text_json, indent=4) 
        text_json = " ".join(text_json)
        self.json[category_name]['products']['details'] = text_json

    def get_spec_prod_details(product_link): 
        r = requests.get(product_link)
        soup = BeautifulSoup(r.content, "html.parser")
        content = soup.find_all(class_='ProductHero__content')
    
        for c in content:
            name = c.find(class_= 'Text-ds Text-ds--body-1 Text-ds--left')
            price = c.find(class_='ProductPricing')
            id_num = c.find(class_='Text-ds Text-ds--body-3 Text-ds--left Text-ds--neutral-600')

            product_price = price.get_text()
            product_name = name.get_text()
            product_id = id_num.get_text()

            self.json[category_name]['products'] = {
                'name': product_name,
                'price': product_price,
                'id': product_id
            }


            
    def get_product_reviews(product_link):
        r = requests.get('https://www.ulta.com/p/double-wear-stay-in-place-foundation-xlsImpprod14641507?sku=2309420')
        soup = BeautifulSoup(r.content, "html.parser")
        review_content = soup.find_all(class_='reviews')
        content = soup.find_all(class_='ProductHero__content')
        reviews = []
 
        for c in content:
            name = c.find(class_= 'Text-ds Text-ds--body-1 Text-ds--left')
            price = c.find(class_='ProductPricing')
            id_num = c.find(class_='Text-ds Text-ds--body-3 Text-ds--left Text-ds--neutral-600')


            #product_link = product_link_tag.get('href')
            product_price = price.get_text()
            product_name = name.get_text()
            product_id = id_num.get_text()
            print(product_name, product_price, product_id)

ModuleNotFoundError: No module named 'jsonpath_ng'

# Scratch Work 

## Extract Product Info to One String

In [1]:
import requests
from bs4 import BeautifulSoup
import html_to_json
import json
from jsonpath_ng import jsonpath, parse

#<a class="Link_Huge Link_Huge--secondary" target="_self" href="https://www.ulta.com/p/double-wear-stay-in-place-foundation-xlsImpprod14641507?sku=2309420"><div class="ProductCard__image ProductCard__image__full__small"><div class="Image"><img class="" alt="Estée Lauder Double Wear Stay-in-Place Foundation" aria-hidden="true" loading="lazy" width="360" height="360" src="https://media.ulta.com/i/ulta/2309420?w=360&amp;$ProductCardNeutralBGLight$&amp;fmt=auto"></div><span class="ProductCard__variant"><p class="Text-ds Text-ds--body-3 Text-ds--left Text-ds--neutral-600">56 colors</p></span></div><div class="ProductCard__content"><h3 class="ProductCard__heading"><span class="ProductCard__brand"><span class="Text-ds Text-ds--body-2 Text-ds--left Text-ds--neutral-600">Estée Lauder</span></span><span class="ProductCard__product"><span class="Text-ds Text-ds--body-2 Text-ds--left">Double Wear Stay-in-Place Foundation</span></span></h3><div class="ProductCard__rating"><div class="ReviewStarsCard"><svg aria-hidden="true" width="12" height="12"><path d="M5.999 1c.136 0 .262.076.324.198L7.81 4.144l2.862.284a.358.358 0 01.222.61L8.54 7.373l.873 3.172a.361.361 0 01-.31.452l-.051.002a.36.36 0 01-.15-.039L5.998 9.524 3.1 10.959a.361.361 0 01-.099.033l-.05.005a.361.361 0 01-.362-.454l.873-3.172-2.357-2.335a.358.358 0 01.222-.61l2.862-.284 1.485-2.944A.364.364 0 015.999 1z" fill="currentColor" fill-rule="nonzero"></path></svg><svg aria-hidden="true" width="12" height="12"><path d="M5.999 1c.136 0 .262.076.324.198L7.81 4.144l2.862.284a.358.358 0 01.222.61L8.54 7.373l.873 3.172a.361.361 0 01-.31.452l-.051.002a.36.36 0 01-.15-.039L5.998 9.524 3.1 10.959a.361.361 0 01-.099.033l-.05.005a.361.361 0 01-.362-.454l.873-3.172-2.357-2.335a.358.358 0 01.222-.61l2.862-.284 1.485-2.944A.364.364 0 015.999 1z" fill="currentColor" fill-rule="nonzero"></path></svg><svg aria-hidden="true" width="12" height="12"><path d="M5.999 1c.136 0 .262.076.324.198L7.81 4.144l2.862.284a.358.358 0 01.222.61L8.54 7.373l.873 3.172a.361.361 0 01-.31.452l-.051.002a.36.36 0 01-.15-.039L5.998 9.524 3.1 10.959a.361.361 0 01-.099.033l-.05.005a.361.361 0 01-.362-.454l.873-3.172-2.357-2.335a.358.358 0 01.222-.61l2.862-.284 1.485-2.944A.364.364 0 015.999 1z" fill="currentColor" fill-rule="nonzero"></path></svg><svg aria-hidden="true" width="12" height="12"><path d="M5.999 1c.136 0 .262.076.324.198L7.81 4.144l2.862.284a.358.358 0 01.222.61L8.54 7.373l.873 3.172a.361.361 0 01-.31.452l-.051.002a.36.36 0 01-.15-.039L5.998 9.524 3.1 10.959a.361.361 0 01-.099.033l-.05.005a.361.361 0 01-.362-.454l.873-3.172-2.357-2.335a.358.358 0 01.222-.61l2.862-.284 1.485-2.944A.364.364 0 015.999 1z" fill="currentColor" fill-rule="nonzero"></path></svg><svg aria-hidden="true" width="12" height="12"><path d="M2.712 10.917a.361.361 0 01-.122-.373l.388-1.41.483-1.762-2.355-2.334a.358.358 0 01.171-.603l.051-.008 2.862-.283L5.675 1.2a.364.364 0 01.274-.196L6.001 1v.002l.049.003a.364.364 0 01.273.195L7.81 4.146l2.862.283a.358.358 0 01.222.61L8.54 7.375l.873 3.172a.361.361 0 01-.511.416L6 9.525 3.103 10.96a.36.36 0 01-.391-.043zM6 2.041v6.817c.078 0 .156.015.23.042l.064.028 2.292 1.135-.69-2.512a.667.667 0 01.129-.602L8.07 6.9l1.889-1.873-2.215-.219a.667.667 0 01-.495-.304l-.034-.059z" fill="currentColor" fill-rule="nonzero"></path></svg><span class="sr-only">4.6 out of 5 stars ; 10645 reviews</span><span class="Text-ds Text-ds--body-3 Text-ds--left Text-ds--neutral-600" aria-hidden="true">(10,645)</span></div></div><div class="ProductCard__price"><div class="ProductPricing"><span class="Text-ds Text-ds--body-2 Text-ds--left Text-ds--black">$52.00 </span></div></div><div class="ProductCard__offers"><div><p class="Text-ds Text-ds--body-3 Text-ds--left Text-ds--magenta-500">Bundle &amp; Save!</p></div></div></div></a>

def get_product_links(): 
    r = requests.get("https://www.ulta.com/shop/makeup/face/foundation")
    soup = BeautifulSoup(r.content, "html.parser")
    content = soup.find_all(class_='ProductCard')
    product_links = []
    for c in content:
        product_link_tag = c.find('a', class_='Link_Huge Link_Huge--secondary')
        product_link = product_link_tag.get('href')
        product_name = product_link_tag.get_text()
        #print(product_name)
        #print(product_link)

        #print(get_product_details(product_link) )


import html2json
import json
from jsonpath_ng import jsonpath, parse

def extract_text_fields(json_data):
    """Extract all text fields using jsonpath-ng."""
    jsonpath_expression = parse('$.._value')
    matches = jsonpath_expression.find(json_data)
    return [match.value for match in matches]

def html_to_text_json(html_content):
    """Convert HTML content to a nested JSON containing only text fields."""
    json_data = html_to_json.convert(html_content)
    text_fields = extract_text_fields(json_data)
    #print(json.dumps(json_data, indent=4))
    return text_fields

# Example HTML content
r = requests.get('https://www.ulta.com/p/double-wear-stay-in-place-foundation-xlsImpprod14641507?sku=2309420')
soup = BeautifulSoup(r.content, "html.parser")
html_content =  soup.find_all(class_='ProductHero__content')
#print(html_content)


# Convert HTML to text-only JSON
text_json = html_to_text_json(str(html_content))

# Print the JSON output
#print(json.dumps(text_json, indent=4) )
print(' '.join(text_json))

ModuleNotFoundError: No module named 'html_to_json'

## Get Specific Product Fields

In [ ]:
import requests
from bs4 import BeautifulSoup
import html_to_json
import json
from jsonpath_ng import jsonpath, parse
#pr_review
def get_product_reviews(product_link):
    r = requests.get('https://www.ulta.com/p/double-wear-stay-in-place-foundation-xlsImpprod14641507?sku=2309420')
    soup = BeautifulSoup(r.content, "html.parser")
    content = soup.find_all(class_='ProductHero__content')
    print(content)
    reviews = []
 
    for c in content:
        name = c.find(class_= 'Text-ds Text-ds--body-1 Text-ds--left')
        price = c.find(class_='ProductPricing')
        id_num = c.find(class_='Text-ds Text-ds--body-3 Text-ds--left Text-ds--neutral-600')

        review_content = soup.find_all(class_='reviews')


        #product_link = product_link_tag.get('href')
        product_price = price.get_text()
        product_name = name.get_text()
        product_id = id_num.get_text()
        #print(product_name, product_price, product_id)



  

print( get_product_reviews('') )  

## Get Review Info

In [ ]:
import requests
from bs4 import BeautifulSoup
import html_to_json
import json
from jsonpath_ng import jsonpath, parse
#pr_review


def extract_text_fields(json_data):
    """Extract all text fields using jsonpath-ng."""
    jsonpath_expression = parse('$.._value')
    matches = jsonpath_expression.find(json_data)
    return [match.value for match in matches]

def html_to_text_json(html_content):
    """Convert HTML content to a nested JSON containing only text fields."""
    json_data = html_to_json.convert(html_content)
    text_fields = extract_text_fields(json_data)
    #print(json.dumps(json_data, indent=4))
    return text_fields


# Example HTML content
r = requests.get('https://www.ulta.com/p/double-wear-stay-in-place-foundation-xlsImpprod14641507?sku=2309420')
soup = BeautifulSoup(r.content, "html.parser")
html_content =  soup.find_all(class_='ProductReviews__Content')
#print(str(html_content))


# Convert HTML to text-only JSON
text_json = html_to_text_json(str(html_content))

# Print the JSON output
#print(json.dumps(text_json, indent=4) )
print(text_json)

In [ ]:
import requests
from bs4 import BeautifulSoup
import html_to_json
import json
from jsonpath_ng import jsonpath, parse
from requests_html import HTMLSession  
import nest_asyncio
import time

from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager

'''
#&pr_rd_page=10
r = requests.get('https://www.ulta.com/p/double-wear-stay-in-place-foundation-xlsImpprod14641507?sku=2309420&pr_rd_page=10')
soup = BeautifulSoup(r.content, "html.parser")
html_content =  soup.find_all(class_='ProductReviews__Content')
print(html_content)
'''
url = 'https://www.ulta.com/p/double-wear-stay-in-place-foundation-xlsImpprod14641507?sku=2309420'
# Setup WebDriver
driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()))
driver.get('file://' + 'temp.html')


with open('temp.html', 'w') as file:
    file.write(html_content)

# Extract the JavaScript code from a specific part of the HTML
soup = BeautifulSoup(html_content, 'html.parser')
content_div = soup.find('div', {'id': 'content'})
scripts = content_div.find_all('script')

javascript_code = []
for script in scripts:
    if script.string:
        javascript_code.append(script.string.strip())

# Execute the JavaScript code
for js in javascript_code:
    driver.execute_script(js)

# Optionally, print the result of JavaScript execution
result = driver.find_element_by_id('result')